In [2]:
import pandas as pd
import numpy as np
#from pandas.io.common import is_url
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import re
import random
from fuzzywuzzy import fuzz
"""import plotly.graph_objs as go
import chart_studio.plotly as py
import plotly.offline as offline
#offline.init_notebook_mode()
import cufflinks as cf
pd.options.display.max_columns = 30
from IPython.core.interactiveshell import InteractiveShell
import plotly.figure_factory as ff
InteractiveShell.ast_node_interactivity = 'all'
from plotly.offline import iplot
cf.go_offline()
cf.set_config_file(world_readable=True, theme='solar')"""



C:\ProgramData\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


"import plotly.graph_objs as go\nimport chart_studio.plotly as py\nimport plotly.offline as offline\n#offline.init_notebook_mode()\nimport cufflinks as cf\npd.options.display.max_columns = 30\nfrom IPython.core.interactiveshell import InteractiveShell\nimport plotly.figure_factory as ff\nInteractiveShell.ast_node_interactivity = 'all'\nfrom plotly.offline import iplot\ncf.go_offline()\ncf.set_config_file(world_readable=True, theme='solar')"

In [4]:
path="D:/scrapping/magicbricks/"
df=pd.read_csv(f'{path}data_noida_feb.csv')

In [5]:
df.description[0]

"8 Bath,Semi-Furnished A Name That Epitomizes Strength & UniquenessThe Logo Focussed on Greenery, Vaastu And Premium Value. The logo has been designed to capture four things mainly: greenery, lots of sunlight, East facing and spaciousness. Besides all these, it has also been designed to capture unfettered, uncluttered living.The tagline also differentiates this property from other such properties in the market by its conceitedness. It reflects confidence of a premium property and the class of people owing such a property.'Sun Twilight' is the name of Project launched by under Company name 'Sunrise Structures & Developers Pvt. Ltd.' The project location is 'Sector 27, Opposite Delta 1 Metro Station, Greater Noida on 105 meter road and other side is 60 meter road. Project is next to City Park, Greater Noida and surrounded by best Commercial Market and Residential Societies."

In [ ]:
print('We have ', len(df), 'properties in the data')

In [ ]:
def print_description(society):
    example = df[df.Society == society][['description', 'name']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Name:', example[1])

In [ ]:
print_description('Mahagun Mantra 2')

In [ ]:
df.Society.value_counts()

In [ ]:
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer().fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]



In [ ]:
df['description']=df['description'].replace('is_url','')

In [ ]:
common_words = get_top_n_words(df['description'], 20)
df1 = pd.DataFrame(common_words, columns = ['description' , 'count'])


In [ ]:
if  df['description'].str.contains('link').any():
    print(df['description'])

In [ ]:
df['description']=df['description'].astype(str)

In [ ]:
del df['url']

In [ ]:
df

In [ ]:
df1.groupby('description').sum()['count'].sort_values().iplot(kind='bar', yTitle='Count', linecolor='black', title='Top 20 words in property description before removing stop words')

In [ ]:
def get_top_n_trigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(1, 3)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [ ]:
common_words = get_top_n_trigram(df['description'], 20)
df5 = pd.DataFrame(common_words, columns = ['description' , 'count'])
df5.groupby('description').sum()['count'].sort_values(ascending=False).iplot(kind='bar', yTitle='Count', linecolor='black', title='Top 20 trigrams in hotel description before removing stop words')


In [6]:
df.floorno.unique()

array([nan, '10', '20', '19', '15', 'Ground', '21', '7', '14', '23', '1',
       '9', '5', '12', '3', '13', '2', '8', '4', '16', '18', '11', '22',
       '30', '17', '6', '24', '28', '25', 'Upper Basement', '27', '29',
       '31', '26'], dtype=object)

In [7]:
df.Society.unique()

array(['Sun Twilight', 'Purvanchal Royal City', 'ATS Destinaire',
       'SKA Metro Ville', 'Arihant Ambar', 'Gaur City',
       'Gaur Yamuna City', 'Prideville ', 'Page Three Residences',
       'Stellar One', 'Rise Resort Residences', 'Ace Platinum',
       'Eros Sampoornam I', 'Gaur Saundaryam', 'SKA Greenarch',
       'Godrej Golf Links', 'ATS Dolce', 'Migsun Ultimo',
       'Sikka Kaamya Greens', 'Gaurs Runway Suites', 'Meridian Towers',
       'Jaypee Greens Villas', 'Paramount Golf Foreste', 'Raksha Addela',
       'Gaur Atulyam', 'Czar Suites', 'Royal Apartments',
       'Supertech Czar Villas', 'ATS Green Paradiso', nan,
       'Supertech Up Country', 'Nimbus Express Park View II',
       'Technocity Apartments', 'Plumeria Garden Estate',
       'Harmukh Apartment', 'NRI City Township', 'Nirala Greenshire',
       'Assotech Springfields', 'The Hemisphere', 'Mahagun Mantra 2',
       'Ajnara Homes', 'Patel Neotown', 'NRI Residency',
       'Ajnara Le Garden', 'Ajnara Panorama',

In [ ]:
def get_top_n_trigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(3, 3), stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [ ]:
common_words = get_top_n_trigram(df['description'], 40)
df6 = pd.DataFrame(common_words, columns = ['description' , 'count'])

In [ ]:
common_words

In [ ]:
df['word_count'] = df['description'].apply(lambda x: len(str(x).split()))
desc_lengths = list(df['word_count'])
print("Number of descriptions:",len(desc_lengths),
      "\nAverage word count", np.average(desc_lengths),
      "\nMinimum word count", min(desc_lengths),
      "\nMaximum word count", max(desc_lengths))

In [ ]:
df['word_count'].iplot(
    kind='hist',
    bins = 50,
    linecolor='black',
    xTitle='word count',
    yTitle='count',
    title='Word Count Distribution in Hotel Description')

In [ ]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
url_symbol=re.compile('[^https?:\/\/.*[\r\n]*]')
STOPWORDS = set(stopwords.words('english'))

In [ ]:
def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text =url_symbol.sub('',text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text

In [ ]:
df['desc_clean'] = df['description'].apply(clean_text)

In [ ]:
common_words = get_top_n_trigram(df['desc_clean'], 20)
df6 = pd.DataFrame(common_words, columns = ['desc_clean' , 'count'])
df6.groupby('desc_clean').sum()['count'].sort_values(ascending=False).iplot(kind='bar', yTitle='Count', linecolor='black', title='Top 20 trigrams in hotel description after removing stop words')


In [ ]:
df6

In [ ]:
df.set_index('name', inplace = True)
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df['desc_clean'])
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
indices = pd.Series(df.index)

In [ ]:
def recommendations(name, cosine_similarities = cosine_similarities):
    
    recommended_properties = []
    
    # gettin the index of the hotel that matches the name
    if indices[indices == name].shape[0]>0:
        idx = indices[indices == name].index[0]
    else:
        idx=clean_text(name)
        tfidf_matrix = tf.fit_transform(idx)
    print(idx)

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar hotels except itself
    top_20_indexes = list(score_series.iloc[1:21].index)
    
    # populating the list with the names of the top 10 matching hotels
    for i in top_20_indexes:
        recommended_properties.append(list(df.index)[i])
        
    return recommended_properties

In [ ]:
recommendations('2 BHK Apartment for Sale in Shri Radha Sky Garden, Noida Extension')

In [ ]:
count = CountVectorizer()
count_vector = count.fit(df['description'])

In [ ]:
count_matrix = count.fit_transform(df['description'])

In [ ]:
print(count_vector.vocabulary_)

In [ ]:
def get_index(reqbased,features):
    indices = pd.Series(reqbased.index)
    score_ind=0
    index_ind=0
    #print(reqbased.head())
    #print(features)
    for index,row  in reqbased.iterrows():
        desc=row['projectName']
        #print(desc)
        score=fuzz.token_set_ratio(desc, features)
        print(score)
        if score>score_ind:
            score_ind=score
            index_ind=index
    print(index_ind)
    print(score_ind)
    return reqbased[reqbased.index==index_ind].index[0]   

In [ ]:
def get_recommedation_budget(locality,budget=None,*features):
    #print
    df['locality']=df['locality'].str.lower()
    reqbased=df[df['locality']==locality.lower()]
    print(reqbased.head())
    if budget!=None:
        budget=int(budget)
        new_budget=budget+(budget*0.10)
        reqbased=reqbased[reqbased['price']<=(new_budget)]
    #reqbased=reqbased.set_index(np.arange(reqbased.shape[0]))
    print(get_index(reqbased,features))
    
    
            
        
    

In [ ]:
get_recommedation_budget('Noida Extension' ,8000000, "3 BHK with 1400 sqft apartment in Greater Noida West")

In [ ]:
df.Society.value_counts()

In [ ]:
temp = re.findall(r'\d+', "3 BHK with minimum 1400 sqft apartment")

In [ ]:
temp

In [ ]:
for n in temp:
    print(len(str(n)))

In [ ]:
import nltk, re, string, collections
from nltk.util import ngrams

In [ ]:
tokenizr="3 BHK with minimum 1400 sqft apartment".split(" ")

In [ ]:
esBigrams = ngrams(tokenizr, 2)

In [ ]:
esBigramFreq = collections.Counter(esBigrams)

In [ ]:
esBigramFreq.keys()

In [ ]:
df['description'] = df['description'].str.replace('[^\w\s]',' ')

In [ ]:
df['description'][0]

In [1]:
common_names = get_top_n_trigram(df.index, 20)

NameError: name 'get_top_n_trigram' is not defined

NameError: name 'df' is not defined